In [1]:
import gym
from gym import Env
from gym.spaces import Box

import pybullet as p
import pybullet_data
from pybullet_utils import bullet_client

import time
import os
import math
import numpy as np
import matplotlib.pyplot as plt

from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import SubprocVecEnv, DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.env_checker import check_env

pybullet build time: Oct 11 2021 20:59:39


In [2]:
RENDER_HEIGHT = 360
RENDER_WIDTH = 480

In [3]:
class Goal:
    def __init__(self, client, base):
        # f_name = os.path.join(os.path.dirname(__file__), 'goal.urdf')
        p.loadURDF(fileName="goal.urdf",
                   basePosition=[base[0], base[1], 0],
                   physicsClientId=client)


In [4]:
class Robot:
    def __init__(self, client, goal_position):
        self.client = client
        self.robot = p.loadURDF(fileName="2links.urdf",
                              basePosition=[-0.3,-0.3,0],
                              physicsClientId=client)
    
        self.arm = 2
        self.endEffector = 3
        
        self.joints_indecies=[1, 2]
        
        self.joint_1_velocity=0
        self.joint_2_velocity=0
        self.np_random, _ = gym.utils.seeding.np_random()
        self.goal_position = goal_position
        random_move_steps = 10
        pj1 = self.np_random.uniform(-90, 90)
        pj2 = self.np_random.uniform(-180, 180)
        
        # print(p.getJointInfo(self.robot, jointIndex=1, physicsClientId=self.client))
        # print(p.getJointInfo(self.robot, jointIndex=2, physicsClientId=self.client))
        # print(p.getJointInfo(self.robot, jointIndex=0, physicsClientId=self.client))
        p.resetJointState(self.robot, 1,
                            targetValue=pj1,
                            physicsClientId=self.client)

        p.resetJointState(self.robot, 2,
                            targetValue=pj2,
                            physicsClientId=self.client)

#         for i in range(0, random_move_steps):
            

#             p.setJointMotorControl2(self.robot, 1,
#                                 p.POSITION_CONTROL,
#                                 targetPosition=pj1,
#                                 physicsClientId=self.client)
        
#             p.setJointMotorControl2(self.robot, 2,
#                                 p.POSITION_CONTROL,
#                                 targetPosition=pj2,
#                                 physicsClientId=self.client)

#             p.stepSimulation(physicsClientId=self.client)
            
#         for i in range(0, random_move_steps):
#             p.setJointMotorControl2(self.robot, 1,
#                                 p.VELOCITY_CONTROL,
#                                 targetVelocity=0,
#                                 physicsClientId=self.client)
        
#             p.setJointMotorControl2(self.robot, 2,
#                                 p.VELOCITY_CONTROL,
#                                 targetVelocity=0,
#                                 physicsClientId=self.client)


#             p.stepSimulation(physicsClientId=self.client)
        
        
    def apply_action(self, action):
        j1, j2 = action
        # self.joint_1_velocity=j1
        # self.joint_2_velocity=j2
        p.setJointMotorControl2(
                bodyIndex=self.robot,
                jointIndex=1,
                controlMode=p.TORQUE_CONTROL,
                force=j1) 
        p.setJointMotorControl2(
                bodyIndex=self.robot,
                jointIndex=2,
                controlMode=p.TORQUE_CONTROL,
                force=j2) 
#         p.setJointMotorControl2(self.robot, 1,
#                                 p.VELOCITY_CONTROL,
#                                 targetVelocity=j1,
#                                 physicsClientId=self.client)
        
#         p.setJointMotorControl2(self.robot, 2,
#                                 p.VELOCITY_CONTROL,
#                                 targetVelocity=j2,
#                                 physicsClientId=self.client)
        
        # p.setJointMotorControlArray(self.robot, self.joints_indecies,
        #                             controlMode=p.VELOCITY_CONTROL,
        #                             targetVelocities=[j1, j2],
        #                             physicsClientId=self.client)
        
    def get_observation(self):
        
        joint1_state = p.getJointState(self.robot, 1, physicsClientId=self.client)
        joint2_state = p.getJointState(self.robot, 2, physicsClientId=self.client)
        
        pj1, vj1, fj1, tj1 = joint1_state
        pj2, vj2, fj2, tj2 = joint2_state
        pj1s = np.sin(pj1)
        pj1c = np.cos(pj1)
        pj2s = np.sin(pj2)
        pj2c = np.cos(pj2)
        return np.array([pj1s, pj1c, pj2s, pj2c, vj1, vj2, tj1 / 100000.0, tj2 / 100000.0, self.goal_position[0], self.goal_position[1]])
        
    
    def get_endeffector_postion(self):
        arm_link_state = p.getLinkState(self.robot, self.endEffector)
        x, y, z = arm_link_state[0]
        
        return x, y
    
    def get_ids(self):
        return self.robot, self.client
    
    def get_base_position(self):
        return p.getBasePositionAndOrientation(self.robot)

In [5]:
class Plane:
    def __init__(self, client):
        # f_name = os.path.join(os.path.dirname(__file__), 'goal.urdf')
        p.loadURDF(fileName="plane.urdf",
                   basePosition=[0, 0, 0],
                   physicsClientId=client)
    

In [6]:
class TwoLinkRobotEnv(Env):
    metadata = {'render.modes': ['human']}
    
    def __init__(self, open_mode=p.DIRECT):
        self.max_x, self.max_y, self.max_z, self.max_vel = 0.75,0.75,0.55, 0.7
        self.action_space = Box(low=np.array([-1, -1]),high=np.array([1, 1]), dtype=np.float64)
        self.observation_space = Box(low=-float('inf'), high=float('inf'), shape=(10, ), dtype=np.float64)
        
        self.isRender = open_mode == p.GUI
        
        self.client = -1
        
        self.robot = None
        self.goal = None
        self.done = False
        self.prev_dist_to_goal = None
        self.prev_vj1 = 0
        self.prev_vj2 = 0
        self.rendered_img = None
        self.render_rot_matrix = None
        self.epsilon = 0.01
        self.num_envs=10
        self.np_random, _ = gym.utils.seeding.np_random()
        self.closed = False
        self.max_number_of_tries = 5000
        self.cost_rate = 0.0000000001
        self._cam_dist = 3
        self._cam_yaw = 0
        self._cam_pitch = -30
        self.time_step = 1./200 #1.0/200
        self._state = self.reset()
        
    
    def step(self, action):
        self.number_of_tries += 1
        # print(action)
        
        self.robot.apply_action(action * 100000)
        p.stepSimulation()
        
        observation = self.robot.get_observation()
        self._state = observation
        
        reward = self.claculate_reward(action)
        
        return observation, reward, self.done, {}
    
    def claculate_reward(self, action):
        
        joint1_state = p.getJointState(self.robot.robot, 1, physicsClientId=self.client)
        joint2_state = p.getJointState(self.robot.robot, 2, physicsClientId=self.client)
        
        pj1, vj1, fj1, tj1 = joint1_state
        pj2, vj2, fj2, tj2 = joint2_state
        
        x, y = self.robot.get_endeffector_postion()
        
        # dist_to_goal = abs(x - self.goal[0]) + abs(y - self.goal[1])
        dist_to_goal = math.sqrt((x - self.goal[0]) ** 2 + (y - self.goal[1]) ** 2)
        
        reward = 0
        
        if abs(dist_to_goal) <= (self.epsilon):
            #check V is zero
            # print(abs(vj1), abs(vj2))
            reward = 10000 - (vj1**2 + vj2 **2) / 10
            self.done = True
            # if abs(vj1) <0.4 and abs(vj2) < 0.4:
            #     reward = 800
            #     self.done = True
            # elif abs(vj1) <3.4 and abs(vj2) < 3.4:
            #     reward = 400
            #     self.done = True
            # elif abs(vj1) <10.4 and abs(vj2) < 10.4:
            #     reward = 200
            #     self.done = True
        else:
            reward -= dist_to_goal / 100.
        
        if self.number_of_tries >= self.max_number_of_tries:
            self.done = True
        t1, t2 = action
        # cost = (5 * ((tj1) ** 2) + (tj2) ** 2) * (self.cost_rate)  ##
        cost = ( 5 * (t1 ** 2) + t2 ** 2 ) / 10000.0
        reward -= cost
        # print(f"cost {cost}")
        # print(f"Reward: {reward}")
        # print(f"D {dist_to_goal}")
        
        return reward
    
    
    def reset(self):
        
        if (self.client<0):
            self.ownsPhysicsClient = True


            if self.isRender:
                  self._p = bullet_client.BulletClient(connection_mode=p.GUI)
            else:
                    self._p = bullet_client.BulletClient()

            self.client = self._p._client
            self._p.configureDebugVisualizer(p.COV_ENABLE_GUI,0)
        
        p.resetSimulation(self.client)
        p.setGravity(0, 0, -9.8)
        p.setAdditionalSearchPath(pybullet_data.getDataPath()) 
        p.setTimeStep(timeStep=self.time_step, physicsClientId=self.client)
        
        Plane(self.client)
        
        
        
#         x = (self.np_random.uniform(0.4, 1.25) if self.np_random.randint(20) % 2 == 1 else
#              self.np_random.uniform(-0.4, -1.25))
        
#         y = (self.np_random.uniform(0.4, 1.25) if self.np_random.randint(20) % 2 == 1 else
#              self.np_random.uniform(-0.4, -1.25))
        
        x = (self.np_random.uniform(0.3, 1.16) if self.np_random.randint(20) % 2 == 1 else
             self.np_random.uniform(-0.3, -1.16))
        
        y = (self.np_random.uniform(0.3, 1.16) if self.np_random.randint(20) % 2 == 1 else
             self.np_random.uniform(-0.3, -1.16))
        self.goal = (x, y)
        self.done = False
        
        self.robot = Robot(self.client, self.goal)
        
        Goal(self.client, self.goal)
        
        observation = self.robot.get_observation()
        
        self.number_of_tries = 0
        
        return observation
    
    def render(self, mode='human'):
        base_pos, _ = self.robot.get_base_position()
        view_matrix = self._p.computeViewMatrixFromYawPitchRoll(
            cameraTargetPosition=base_pos,
            distance=self._cam_dist,
            yaw=self._cam_yaw,
            pitch=self._cam_pitch,
            roll=0,
            upAxisIndex=2)
        proj_matrix = self._p.computeProjectionMatrixFOV(fov=60,
                                                                       aspect=float(RENDER_WIDTH) / RENDER_HEIGHT,
                                                                       nearVal=0.1,
                                                                       farVal=100.0)
        (_, _, px, _, _) = self._p.getCameraImage(
            width=RENDER_WIDTH,
            height=RENDER_HEIGHT,
            renderer=self._p.ER_BULLET_HARDWARE_OPENGL,
            viewMatrix=view_matrix,
            projectionMatrix=proj_matrix)
        rgb_array = np.array(px)
        rgb_array = rgb_array[:, :, :3]
        return rgb_array
    
    def close(self):
        self.closed = True
        p.disconnect(self.client)

In [7]:
# check_env(TwoLinkRobotEnv())

In [8]:
def make_env(id):
    def _init():
        if id == 1:
            env = TwoLinkRobotEnv(p.GUI)
        else :
            env = TwoLinkRobotEnv()
        return Monitor(env, "env_logs/2rr_logs_" + str(id) + ".csv")
    return _init

number_of_cpu = 18
env = SubprocVecEnv([make_env(i) for i in range(number_of_cpu)])
# env = DummyVecEnv([lambda:Monitor(TwoLinkRobotEnv(p.GUI))])
# episodes = 5
# for episode in range(1, episodes+1):
#     state = env.reset()
#     done = False
#     score = 0 
    
#     while not done:
#         action = env.action_space.sample()
#         n_state, reward, done, info = env.step(action)
#         score+=reward
#     print('Episode:{} Score:{}'.format(episode, score))
# env.close()


pybullet build time: Oct 11 2021 20:59:39
pybullet build time: Oct 11 2021 20:59:39
pybullet build time: Oct 11 2021 20:59:39
pybullet build time: Oct 11 2021 20:59:39
pybullet build time: Oct 11 2021 20:59:39
pybullet build time: Oct 11 2021 20:59:39
pybullet build time: Oct 11 2021 20:59:39


In [9]:
log_path = os.path.join("2rr_logs")

In [10]:
net_arch=[dict(pi=[36, 18], vf=[36, 18])]

In [11]:
model = PPO("MlpPolicy", env, verbose=1, tensorboard_log=log_path, policy_kwargs={'net_arch': net_arch})

Using cpu device


In [ ]:
model.set_env(env)
model.learn(total_timesteps=30000000)

pybullet build time: Oct 11 2021 20:59:39
pybullet build time: Oct 11 2021 20:59:39
pybullet build time: Oct 11 2021 20:59:39
pybullet build time: Oct 11 2021 20:59:39
pybullet build time: Oct 11 2021 20:59:39
pybullet build time: Oct 11 2021 20:59:39
pybullet build time: Oct 11 2021 20:59:39
pybullet build time: Oct 11 2021 20:59:39
pybullet build time: Oct 11 2021 20:59:39
pybullet build time: Oct 11 2021 20:59:39
pybullet build time: Oct 11 2021 20:59:39
2021-11-05 00:01:25.270058: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/rosim/.local/lib/python3.8/site-packages/cv2/../../lib64:/usr/lib/x86_64-linux-gnu/gazebo-11/plugins:/opt/ros/foxy/opt/yaml_cpp_vendor/lib:/opt/ros/foxy/opt/rviz_ogre_vendor/lib:/opt/ros/foxy/lib/x86_64-linux-gnu:/opt/ros/foxy/lib:/home/rosim/.mujoco/mujoco200/bin
2021-11-05 00:01:25

Logging to 2rr_logs/PPO_88
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1.11e+03 |
|    ep_rew_mean     | 9.73e+03 |
| time/              |          |
|    fps             | 6487     |
|    iterations      | 1        |
|    time_elapsed    | 5        |
|    total_timesteps | 36864    |
---------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 1.11e+03     |
|    ep_rew_mean          | 9.73e+03     |
| time/                   |              |
|    fps                  | 3586         |
|    iterations           | 2            |
|    time_elapsed         | 20           |
|    total_timesteps      | 73728        |
| train/                  |              |
|    approx_kl            | 0.0044943765 |
|    clip_fraction        | 0.0291       |
|    clip_range           | 0.2          |
|    entropy_loss         | -2.84        |
|    explained_variance   | 1.6

In [ ]:
model.save("two_links_model_3m_reward_final")

In [ ]:
# evaluate_policy(model, env, n_eval_episodes=50, render=False)
# env.close()

In [ ]:
# new_env = TwoLinkRobotEnv(p.GUI)
# model = PPO.load("two_links_model_3m_new_reward_2")
# evaluate_policy(model, new_env, n_eval_episodes=20, render=True)

In [ ]:
# new_env.close()